# Hash the Trash notebook

Here everything should run smoothly with just this notebook, running Ganache (with quickstart) and setting the PORT on which Ganache is running (The contract's abi and bytecode has been compiled from Remix and imported here to run everything)

Various sections are:
* [deploying the contract](#deploy)
* [operating with its functions from eth accounts](#functions)
    * [TARI](#tari)
    * [Trash cycle](#cycle)
    * [Refund](#refund)
* [picking up logs of certain events from the chain](#logs)

In [1]:
# mandatory imports
import random
import pandas as pd
from web3 import Web3
from abi_bytecode import abi, bytecode # saved externally as .py
from bs4 import BeautifulSoup
import time, requests, json
from forex_python.converter import CurrencyRates

In [2]:
# Connecting to ganache through opened up PORT
ganache_url = 'HTTP://127.0.0.1:7545'      #change here if different
web3 = Web3(Web3.HTTPProvider(ganache_url))
web3.isConnected()

True

In [4]:
# Compiled abi and bytecode of trash.sol which inherits from citizenz.sol (and Ownable, safemath etc)
abiRemix = json.loads(abi)         # turned to string after copy to clipboard from Remix
bytecodeRemix = bytecode['object'] # it is a dictionary (as copy to clipboard form remix), we use the object for web3 deployment

# simple database example and adding column with ganache accounts
data = pd.read_excel('data/agents_data.xlsx')
data['address'] = web3.eth.accounts     
data

,name,surname,family,mq,weight,recycle,lat,long,role,address
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0x80b4eD803171D79F024887c51815deBeBe0c2009
1,Francesca,Bianchessi,2.0,70.0,500.0,NaN,NaN,NaN,citizen,0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
2,Ilaria,Bolla,4.0,200.0,900.0,NaN,NaN,NaN,citizen,0x93B46B648097Fc5e26c6007938fC401923AEACB6
3,Alessandro,Botti,5.0,340.0,800.0,NaN,NaN,NaN,citizen,0xDF6C81c1aA61c6a778CDC086106A9677a3ba625A
4,Davide,Castellini,3.0,50.0,200.0,NaN,NaN,NaN,citizen,0x38e6F3f677540be1BE25a6fCda7ACE492D23dbe5
5,Anna,Di Marco,4.0,80.0,560.0,NaN,NaN,NaN,citizen,0xa1758d77168BC9b14BD161c5ac054618121FA52C
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56
8,Disposal Station,Milano,NaN,NaN,NaN,True,45.4642,9.1900,station,0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9
9,Disposal Station,Bologna,NaN,NaN,NaN,False,44.4949,11.3426,station,0x4C70C3653FAED0E3AF4E3d4388d21392338Bb3C0


<a name = 'deploy'></a>

### Deploying the contract

Here we first create a dictionary with the readily available 10 accounts from ganache and then make the municiaplity deploy the contract on the blockchain

In [5]:
def deploy_contract(deployer, _abi, _bytecode):
    """
    Deploy the contract using Python and web3.py without relying on Remix (aside from getting the compiled abi and bytecode)
    
    Parameters
    -------------
        deployer: eth account
        abi, bytecode : compiled contract things
        
    Returns
    -------------
        contract instance
    """
    contract = web3.eth.contract(abi=_abi, bytecode=_bytecode)      # compiled contract
    tx_hash = contract.constructor().transact({'from': deployer}) # contract constructor call (i.e. deploy)
    
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)      # Get receipt when deployed to blockchain
    print(f"Deployed contract at address:\n{tx_receipt.contractAddress}") # contract address
    
    deployed_ctr = web3.eth.contract(address = tx_receipt.contractAddress, abi  = _abi) # contract
    
    return deployed_ctr

In [6]:
# Deploy contract (municipality in our case) [can check from ganache the blocks to see if it happened]

contract = deploy_contract(deployer = data[data.role == 'municipality']['address'].item(), _abi = abiRemix, _bytecode = bytecodeRemix)

Deployed contract at address:
0x77E754bA0BEC402Edad8767321ab313915cd4EC9


<a name = 'functions'></a>
## Interacting with the functions

First a simplle check of the owner of the contract and then creating the Agents.

(Still unsure but i think function(...).**call**(...) works only for 'view' functions; need function(...).**transact**(...) for those that modify the chain)

In [7]:
# simple check for owner of the contract
owner = contract.functions.owner().call() # get owner from contract function
print(f"owner: {owner}")
print(f"Is it the municipality? {owner == data[data.role == 'municipality']['address'].item()}")

# to remember inputs and all functions
contract.all_functions()

owner: 0x80b4eD803171D79F024887c51815deBeBe0c2009
Is it the municipality? True


[<Function MunicipalityBalance()>,
 <Function TariAmount(address)>,
 <Function citizens(address)>,
 <Function createCitizen(address,string,uint256,uint256,uint256)>,
 <Function createStation(address,bool,int256,int256)>,
 <Function createTruck(address,bool)>,
 <Function deleteCitizen(address)>,
 <Function deleteStation(address)>,
 <Function deleteTruck(address)>,
 <Function drop(address,int256,int256)>,
 <Function givePayout(address)>,
 <Function numberC()>,
 <Function numberS()>,
 <Function numberT()>,
 <Function owner()>,
 <Function payTari()>,
 <Function pick(address,uint256,uint256)>,
 <Function received(bool,address,uint256)>,
 <Function renounceOwnership()>,
 <Function stations(address)>,
 <Function transferOwnership(address)>,
 <Function trucks(address)>,
 <Function withdraw()>]

In [8]:
# Creating list of touples as correct inputs for the contract structs

# Create CITIZENS - (address payable _address, string memory _name, uint _family, uint _house, uint256 _w)
citizensL = [[r.address, " ".join([r.name, r.surname]), int(r.family), int(r.mq), int(r.weight)] 
             for r in data.itertuples() if r.role == 'citizen']

# Create TRUCKS - (address _address, bool _recycle)
trucksL = [[r.address, r.recycle] for r in data.itertuples() if r.role == 'truck']

# Create STATIONS - (address _address, bool _recycle, int _lat, int _long)
stationsL = [[r.address, r.recycle, int(r.lat), int(r.long)] for r in data.itertuples() if r.role == 'station']

print(f'Example of createCitizen inputs: {citizensL[0]}')

Example of createCitizen inputs: ['0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10', 'Francesca Bianchessi', 2, 70, 500]


In [9]:
def create(municip_addr ,c, t, s):
    '''
    Calls contract funcions to assigns addresses (and their characteristics) roles and populate mappings
    
    Parameters
    ----------------------
        municip_addr : eth address of the municipality
        c, t, s : list of touples with correct characteristics
        
    Returns
    ----------------------
        nothing on python, the chain grows as these transactions are registered
    '''
    from_dict = {'from': municip_addr}
    for i in range(len(c)):
        contract.functions.createCitizen(c[i][0], c[i][1], c[i][2], c[i][3], c[i][4]).transact(from_dict)
    for j in range(len(t)):
        contract.functions.createTruck(t[j][0], t[j][1]).transact(from_dict)
    for k in range(len(s)):
        contract.functions.createStation(s[k][0], s[k][1], s[k][2], s[k][3]).transact(from_dict)
    print('Done')

In [10]:
# Populating mappings and check amount of entities
create(data[data.role == 'municipality']['address'].item(),citizensL, trucksL, stationsL)

print(f"""
# Citizens: {contract.functions.numberC().call()} 
# Trucks : {contract.functions.numberT().call()}
# PLants : {contract.functions.numberS().call()}""")

Done

# Citizens: 5 
# Trucks : 2
# PLants : 2


In [11]:
# Some functions to show structs statuses as we change them
def get_citizen_info(address):
    """
    Call to the contract and retrive (pretty) informations on citizen
    """
    info = contract.functions.citizens(address).call()
    print(f"""
    Address eth : {address}
    Name : {info[0]}
    Family members : {info[1]}
    House sq.meters : {info[2]}
    Assigned weight\liters : {info[3]}
    TARI amount : {info[4]}
    Recyclable Waste Tot : {info[5]}
    Non Recyclable Waste Tot : {info[6]}
    Paid TARI : {info[7]}
    Active account : {info[8]}
    """)

def get_truck_info(address):
    """Show pretty info on trucks"""
    info = contract.functions.trucks(address).call()
    print(f"""
    Address eth : {address}
    Truck number : {info[0]}
    Weight transported : {info[1]}
    Recyclable Truck : {info[2]}
    Active Truck : {info[3]}
    """)
    
    
def get_station_info(address):
    """Show pretty info for stations"""
    info = contract.functions.stations(address).call()
    print(f"""
    Address eth : {address}
    Station nr. : {info[0]}
    Weight : {info[1]}
    latitude : {info[2]}
    longitude : {info[3]}
    Recyclable Plant : {info[4]}
    Active Plant : {info[5]}
    """)

In [12]:
# using exemplar citizen to do check along the road
dutiful_citizen = data[data.role == 'citizen']['address'].reset_index(drop = True)[0]
get_citizen_info(dutiful_citizen)

ex_truck = data[data.role == 'truck']['address'].reset_index(drop = True)[0]
get_truck_info(ex_truck)

ex_station = data[data.role == 'station']['address'].reset_index(drop = True)[0]
get_station_info(ex_station)


    Address eth : 0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 0
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    

    Address eth : 0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    

    Address eth : 0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


<a name = 'tari'></a>
### Compute TARI 

In [13]:
# Calculate amount of TARI for each citizen (and looping for the others)
municipality = data[data.role == 'municipality']['address'].item()

for addr in data[data.role == 'citizen']['address']:
    contract.functions.TariAmount(addr).transact({'from' : municipality})

In [14]:
get_citizen_info(data[data.role == 'citizen']['address'].reset_index(drop = True)[0])
"""looks expensive"""


    Address eth : 0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 500
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : False
    Active account : True
    


'looks expensive'

In [15]:
# Each citizen pays its due amount
for addr in data[data.role == 'citizen']['address']:
    
    info = contract.functions.citizens(addr).call() # need the amount from the contract
    print(f"Citizen {info[0]} is paying {info[4]} wei")
    
    # payable function, the amount is passed in the dictionary of .transact()
    contract.functions.payTari().transact({'from': addr, 'value' : info[4]})
    

Citizen Francesca Bianchessi is paying 190000000000000000 wei
Citizen Ilaria Bolla is paying 490000000000000000 wei
Citizen Alessandro Botti is paying 1440000000000000000 wei
Citizen Davide Castellini is paying 120000000000000000 wei
Citizen Anna Di Marco is paying 216000000000000000 wei


In [16]:
get_citizen_info(data[data.role == 'citizen']['address'].reset_index(drop = True)[0])
"FINALLY PAID"


    Address eth : 0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 0
    Non Recyclable Waste Tot : 0
    Paid TARI : True
    Active account : True
    


'FINALLY PAID'

In [17]:
# Save the TARI amount for each citizen 
tari_list = [contract.functions.citizens(addr).call()[4]*(10**(-18)) 
             for addr in data[data.role == 'citizen']['address']]
data.loc[data.role == 'citizen', 'TARI_eth'] = tari_list

# Convert to EUR 
cmc = requests.get('https://coinmarketcap.com/currencies/ethereum/markets/')
soup = BeautifulSoup(cmc.content, 'html.parser')  
data_coinmkt = soup.find('script', type="application/ld+json")
data_coinmkt = json.loads(data_coinmkt.contents[0])
usd_eth = data_coinmkt['currentExchangeRate']['price']

c = CurrencyRates()
usd_eur_rate = c.get_rate('USD', 'EUR') 
eur_eth = usd_eth * usd_eur_rate 

data.loc[data.role == 'citizen', 'TARI_eur'] = data.loc[data.role == 'citizen', 'TARI_eth']*eur_eth 
data

,name,surname,family,mq,weight,recycle,lat,long,role,address,TARI_eth,TARI_eur
0,Comune,Codogno,0.0,0.0,0.0,NaN,NaN,NaN,municipality,0x80b4eD803171D79F024887c51815deBeBe0c2009,NaN,NaN
1,Francesca,Bianchessi,2.0,70.0,500.0,NaN,NaN,NaN,citizen,0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10,0.190,86.558285
2,Ilaria,Bolla,4.0,200.0,900.0,NaN,NaN,NaN,citizen,0x93B46B648097Fc5e26c6007938fC401923AEACB6,0.490,223.229262
3,Alessandro,Botti,5.0,340.0,800.0,NaN,NaN,NaN,citizen,0xDF6C81c1aA61c6a778CDC086106A9677a3ba625A,1.440,656.020689
4,Davide,Castellini,3.0,50.0,200.0,NaN,NaN,NaN,citizen,0x38e6F3f677540be1BE25a6fCda7ACE492D23dbe5,0.120,54.668391
5,Anna,Di Marco,4.0,80.0,560.0,NaN,NaN,NaN,citizen,0xa1758d77168BC9b14BD161c5ac054618121FA52C,0.216,98.403103
6,Truck,Recycle,NaN,NaN,NaN,True,NaN,NaN,truck,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,NaN,NaN
7,Truck,Not Recycle,NaN,NaN,NaN,False,NaN,NaN,truck,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56,NaN,NaN
8,Disposal Station,Milano,NaN,NaN,NaN,True,45.4642,9.1900,station,0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9,NaN,NaN
9,Disposal Station,Bologna,NaN,NaN,NaN,False,44.4949,11.3426,station,0x4C70C3653FAED0E3AF4E3d4388d21392338Bb3C0,NaN,NaN


<a name = 'cycle'></a>
## Working with trashbags lifecycles

The trucks will pick them up and drop them at the disposal station. From excel files we simulate what the sensors would use as inputs.

In [19]:
bags = pd.read_excel("data/bags_data.xlsx")
bags.head(10) 

,name,surname,weight,recycle
0,Francesca,Bianchessi,1,True
1,Ilaria,Bolla,2,False
2,Alessandro,Botti,3,True
3,Davide,Castellini,1,False
4,Anna,Di Marco,2,True
5,Francesca,Bianchessi,1,True
6,Ilaria,Bolla,6,False
7,Alessandro,Botti,4,True
8,Davide,Castellini,3,False
9,Anna,Di Marco,2,True


#### Pick up trash

Function called from the truck and using sensors to check the weight 

Here we automatically assign the correct truck to the trash type. 
##### ! if the two don't match we have a slight problem !

In [20]:
# The right truck will pick up the various trashbags
for i, name, sur, w, recyclable in bags.itertuples():

    # get address of generator
    generator_addr = data[data.name == name]['address'].item()
    
    # get correct truck address via subsetting
    truck_addr = data[(data['name'] == 'Truck') & (data['recycle'] == recyclable)]['address'].item()
    
    # 'pick' function in contract is called by the correct truck
    contract.functions.pick(generator_addr, w, random.randint(0, 1e+10)).transact({'from' : truck_addr})

In [21]:
get_citizen_info(dutiful_citizen)
get_truck_info(ex_truck)

"""See both counters increased"""


    Address eth : 0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
    Name : Francesca Bianchessi
    Family members : 2
    House sq.meters : 70
    Assigned weight\liters : 0
    TARI amount : 190000000000000000
    Recyclable Waste Tot : 16
    Non Recyclable Waste Tot : 12
    Paid TARI : True
    Active account : True
    

    Address eth : 0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c
    Truck number : 1
    Weight transported : 71
    Recyclable Truck : True
    Active Truck : True
    


'See both counters increased'

#### Drop bags at station

Still a function from the side of the truck. The truck will result empty after dropping and the station will increase its total counter with the recieved weight.

In [23]:
gps = pd.read_excel("data/gps_data.xlsx")
gps

,name,surname,lat,long
0,Disposal Station,Milano,45.4642,9.1900
1,Disposal Station,Bologna,44.4949,11.3426


In [24]:
get_station_info(ex_station)


    Address eth : 0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9
    Station nr. : 1
    Weight : 0
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    


In [25]:
# get station name and coords from the gps
for i, name, sur, lat, long in gps.itertuples():
    
    #get station address
    station_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    
    #get_station_info(station_addr)
    # need to get the type of station (recycling/not recycling) to pair the truck
    s_info = contract.functions.stations(station_addr).call() # s_info[4] is the type
    
    # pairing truck based on recyclable or not and calling the 'drop' function
    correct_truck = data[(data.name == 'Truck') & (data.recycle == s_info[4])]['address'].item()
    
    contract.functions.drop(station_addr, int(lat), int(long)).transact({'from' : correct_truck})
    

In [26]:
get_station_info(ex_station)
get_truck_info(ex_truck)


    Address eth : 0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9
    Station nr. : 1
    Weight : 71
    latitude : 45
    longitude : 9
    Recyclable Plant : True
    Active Plant : True
    

    Address eth : 0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c
    Truck number : 1
    Weight transported : 0
    Recyclable Truck : True
    Active Truck : True
    


#### Recieved 

Function called by the station. Its sensors will say how much they actually recieved and from which truck.(This excel is from the point of view of a single station, just to mess it up on purpose.)

##### ! If the weight detected is not the same as it was in the truck it reverts. This is a problem for the next truck that stops there; since it checks the current total weight at the station which is always higher after the first truck !

In [27]:
station_sense = pd.read_excel("data/sensors_data.xlsx")
station_sense

,name,surname,weight
0,Truck,Recycle,71
1,Truck,Not Recycle,61


In [28]:
for i, name, sur ,weight in station_sense.itertuples():
    
    # get truck address
    truck_addr = data[(data.name == name) & (data.surname == sur)]['address'].item()
    info_tr = contract.functions.trucks(truck_addr).call() # to get truck type in info_tr[2]
    
    
    # station address is the same both times [can change True with info_tr[2] to get the correct station to read the correct truck]
    st_addr = data[(data.name == 'Disposal Station') & (data.recycle == True)]['address'].item() 
    print(f" Truck: {truck_addr}\nStation: {st_addr}")
    
    #recieved function call
    contract.functions.received(info_tr[2], truck_addr, weight).transact({'from' : st_addr})
    
    

 Truck: 0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c
Station: 0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9
 Truck: 0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56
Station: 0xf5cD10ceDdC13bf43d26E9FD22f7782284e6e3D9


SolidityError: execution reverted: VM Exception while processing transaction: revert

<a name = 'refund'></a>
## Refund

municipality withdraws form the contract the ETH and then does the payout for all citizens


In [29]:
print(municipality) # we have the municipality address stored from before

# the withdraw works just like this, see ganache '0' account --> municipality
before = web3.eth.getBalance(municipality)
contract.functions.withdraw().transact({'from' : municipality}) 
after = web3.eth.getBalance(municipality)
print(f"Withdrawn: {after-before} (also minus gas costs)")

0x80b4eD803171D79F024887c51815deBeBe0c2009
Withdrawn: 2455404660000000000 (also minus gas costs)


In [30]:
# citizen address as input to function
for citizen in data[data.role == 'citizen']['address']:
    
    # balance comparison for the refund
    before = web3.eth.getBalance(citizen)
    contract.functions.givePayout(citizen).transact({'from' : municipality})
    after = web3.eth.getBalance(citizen)
    
    print(f"{citizen}\nBefore: {before}\nAfter : {after}\nRefunded:{after-before}")
        

0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10
Before: 99428707180000000000
After : 99428707180000000000
Refunded:0
0x93B46B648097Fc5e26c6007938fC401923AEACB6
Before: 98528707180000000000
After : 98528707180000000000
Refunded:0
0xDF6C81c1aA61c6a778CDC086106A9677a3ba625A
Before: 95678707180000000000
After : 95678707180000000000
Refunded:0
0x38e6F3f677540be1BE25a6fCda7ACE492D23dbe5
Before: 99638707180000000000
After : 99638707180000000000
Refunded:0
0xa1758d77168BC9b14BD161c5ac054618121FA52C
Before: 99350707180000000000
After : 99350707180000000000
Refunded:0


In [31]:
## Right now givePayout isn't really working

<a name = 'logs'></a>

## Event/Log Filtering

After running the above cells the chain is populated with a lot of different logs from events. They can be extracted with specific filters that are contract specific. 

(Right now just 1 filter for the sake of demonstration. A ***real time*** version that repeatedly checks the last block still needs working)

In [32]:
# creating logs/event filters to inspect the whole chain (fromBlock = 1)

# filter for the Event 'PickedUp' generated by the truck
pickedUp_filter = contract.events.PickedUp.createFilter(fromBlock = 1)
#recieved_filter etc

filters = [pickedUp_filter]

In [33]:
def get_past_logs(filter_list):
    """
    Iterates over every filter built and extracts logs from the block specified in the filter to the 'latest'
    
    inputs
    ---------------
     filter_list : filters created
    
    returns
    ---------------
     list containing every event attribute generated from the 'emit' on the contract
    """
    events = []
    for event_filter in filter_list:
        for e in event_filter.get_all_entries(): # get_new_entry() to check only last block
            
            # e is a nested dictionary, like this we bring all elements on the same level
            args = dict(e['args'])
            args.update(dict(e))
            del args['args'] # brought one level above, delete the nest
            # args.pop('args', None) # could delete like this too
            
            events.append(args)
            
    return events

In [34]:
# Pass filters on the chain (from block 1) and pick logs found (for now only 'ToPickUp')
# Can see logs of the 5 citizens, each one a different generator and all the garble generated by the event
events = get_past_logs(filters)
df = pd.DataFrame(events)
df

,transporter,wasteType,bagId,generator,wasteWeight,pickUpTime,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber
0,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,"b""2\xc6\xafS]\xe7q\x8b\xd2\xe1>#z\xd1\n4\xae\x...",0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10,1,1607698600,PickedUp,0,0,b'\xc8]\x96a[Z\xd0\xb6\xc1\xa9\xa9q\xf4\xd9\xf...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\xdeu}kk\x0c>-\x95\x07\xd0\x8cT\xea8\x95\xf9...,151
1,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56,False,b'\xef\x14\xb5t\x868\x1f\xcd\xb8425\xa1\xfcK0\...,0x93B46B648097Fc5e26c6007938fC401923AEACB6,2,1607698601,PickedUp,0,0,b'0|e\x11vALF\xcd\x91\xd6tMk>\xc8\x97\x96\x81U...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\x1e\t\xc3\xadY\xe0\\]\xdd\xf3\x9d\x8bDz\xe1...,152
2,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,b'o\x1dqA\x90\x06\xcf\x1e\xe6\xcd\xb2+^<\xbc\x...,0xDF6C81c1aA61c6a778CDC086106A9677a3ba625A,3,1607698601,PickedUp,0,0,b'\x08[\xa3\x16\xb5\x89\nlp\x0fM\xfam\xb2_\x89...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\xd0\x13\x00\x90g\x84\x0b\xea\xd6\xb7\x81j\x...,153
3,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56,False,b'\xd0\x9b`\xc1g\x8590\xf5\x1e\xc7\x15\xcdZ;\x...,0x38e6F3f677540be1BE25a6fCda7ACE492D23dbe5,1,1607698601,PickedUp,0,0,b'\xce\xa0\xc27O\x8bk\xdb\xd6\xc5\x02\x81\x80W...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\x9c\xf8\xb1a\x1b\xba\xa6\x08\xbe\xa8\xa6$\x...,154
4,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,"b""#\xac\xb4'\x06&\x05[\xa5\x15\x8f\r&\xb5\x06K...",0xa1758d77168BC9b14BD161c5ac054618121FA52C,2,1607698601,PickedUp,0,0,b'6\x10\x80\x80>\x16\xd0e\x0e\x07\xcf\x01\n\x9...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'O\xf1\xc6p\r\xd26D.T\xe8\x80\x1d\xf4Q\xc2\xd...,155
5,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,"b""S`\t\xfa\x8d\x9fy\r\x87R\xa1N\xa0;\x87J\xcc_...",0xD06c293a29E2BB94624bB8b7eFb8003Cf74A2f10,1,1607698602,PickedUp,0,0,"b""\x81gw\xfe\xe14\x0cW\x07\x81\xee\xe9\x0e\xe5...",0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\x93\x9c%@\x89m\xce\xd0\xfe\xe8\x11\x0e\xfft...,156
6,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56,False,b'\xd1\x01YA\xd5f/w\x1b\xcd\xd1h\xf3\xcfm\xf6\...,0x93B46B648097Fc5e26c6007938fC401923AEACB6,6,1607698602,PickedUp,0,0,b'\xd1\xf6\x8d;^\x97\xb4\xb84\xb2<\xd8g\xdf\xa...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'm\x95@T\x11$\xf0\x16y\xb3\x8c\x14\n\x94\xb2\...,157
7,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,"b""l\xbb#\x94\xf7\xdc\xdaw:'\x80\x98D+.\x86\xa8...",0xDF6C81c1aA61c6a778CDC086106A9677a3ba625A,4,1607698602,PickedUp,0,0,b'4s\xaa?c\xdeT\xaa\x91)~\xc02\xaa\xb8i\x9eIP\...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\xfb\xd5c\xfe\x9e\x00\xdad\xc9\xeb\xb0a\x0e\...,158
8,0x66afF8934EF0f0bbEa36eAd08b24F917dd422e56,False,b'x\xc2\x1d\xba3\x10\x85\xc4\xa6+\xa6bn\xcc\xf...,0x38e6F3f677540be1BE25a6fCda7ACE492D23dbe5,3,1607698603,PickedUp,0,0,b'4\xf1\x16\xd3\xdb\xdc\xf0\x10z\x9b\x13\xac\x...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,"b'\xd4\xc2\x0c\xd5\x86`\x7f""M\'\xccc\t\xbe)\xe...",159
9,0xeF479de51f905b288b76d557d9ea66b6B1DfeC9c,True,"b""_\x95\x8e\x91k4'Y%\xa2xX\x11\xef-\x89T\x0f\x...",0xa1758d77168BC9b14BD161c5ac054618121FA52C,2,1607698603,PickedUp,0,0,b'\x8e\xf2$\x05\r.\xfe\xde$\xc4-H\xe8\xd3\xde\...,0x77E754bA0BEC402Edad8767321ab313915cd4EC9,b'\xb2\x8a\xae\xc4\xee\x1f^&Ur\x1f\t\xdaMW\xa0...,160
